# Exploratory notebook for IMACLIM within MATMat-Trade

## Creating the MatMat-Trade model base on IMACLIM regions and sectors but on EXIOBASE 3 data

In [1]:
# make the necessary imports
import tqdm
import shutil
import warnings
import pandas as pd
import numpy as np
import multiprocessing


from main import Model
from src.utils import footprint_extractor
from src.scenarios import DICT_SCENARIOS
from src.advance import extract_data

In [2]:
# for some reason saved model has issues with the names of regions so we rebuild the model each times


shutil.rmtree('data/models/2015__pxp__IMACLIM_format__ghg')

In [3]:


model = Model(aggregation_name="IMACLIM_format")

#ignore performace warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

Loading data... (may take a few minutes)
Data loaded successfully !


## Creating the scenarios based on IMACLIM scenarios and the projected changes 

In [4]:

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

for scenario in tqdm.tqdm(scenario_list[:1]):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)

  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

## Getting the emissions per scenario 

### Single process version, might be long to run 

In [ ]:

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).sort_index()
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]

production_based_emissions=pd. concat([ pd.concat([ model.iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"]for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]) for scenario in scenario_list],
                                        keys=scenario_list,
                                        names=("scenario","region","sector"),
                                        axis=0).sort_index()


for scenario in tqdm.tqdm(scenario_list):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario=scenario,new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    Emissions.loc[(scenario),"2050"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    Emissions.loc[(scenario),"2050-Prod"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    Emissions.loc[(scenario),"2050-Cons"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    production_based_emissions.loc[(scenario),"2050-Cons"]=model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    production_based_emissions.loc[(scenario),"2050-Prod"]=model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    production_based_emissions.loc[(scenario),"2050"]=model.counterfactuals["IMACLIM_all_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    

### Multiple Process versio, should be faster

In [4]:

NBR_PROCESS=4

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).sort_index()
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]

production_based_emissions=pd. concat([ pd.concat([ model.iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"]for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]) for scenario in scenario_list],
                                        keys=scenario_list,
                                        names=("scenario","region","sector"),
                                        axis=0).sort_index()


def task(scenario):
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario=scenario,new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    ret1=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret2=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret3=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret4=model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret5=model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret6=model.counterfactuals["IMACLIM_all_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    return ret1,ret2,ret3,ret4,ret5,ret6


pool = multiprocessing.Pool(processes=NBR_PROCESS)
result_list=pool.map(task,scenario_list)

for i in range(len(scenario_list)):
    scenario=scenario_list[i]
    Emissions.loc[(scenario),"2050"],Emissions.loc[(scenario),"2050-Prod"],Emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Prod"],production_based_emissions.loc[(scenario),"2050"]=result_list[i]
        
    

/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
/tmp/ipykernel_171845/2343872101.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  model.modify_counterfactual("IMACLIM_all_2050_"+sc

Problems on sums of columns  region  sector               
AFR     Terrestrial transport     47.043673
MDE     Terrestrial transport    146.430499
RAL     Terrestrial transport     25.875496
dtype: float64


/tmp/ipykernel_171845/2343872101.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
/tmp/ipykernel_171845/2343872101.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)


Problems on sums of columns  region  sector               
AFR     Terrestrial transport     333.209566
MDE     Terrestrial transport    4486.348277
RAL     Terrestrial transport     172.151602
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     237.323465
MDE     Terrestrial transport    3148.991362
RAL     Terrestrial transport     124.123202
dtype: float64


/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
/tmp/ipykernel_171845/2343872101.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)


Problems on sums of columns  region  sector               
AFR     Terrestrial transport     282.256477
MDE     Terrestrial transport    6422.740342
RAL     Terrestrial transport     173.155012
dtype: float64


/tmp/ipykernel_171845/2343872101.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)


Problems on sums of columns  region  sector               
AFR     Terrestrial transport     169.042045
MDE     Terrestrial transport    2024.383010
RAL     Terrestrial transport      89.719931
dtype: float64


/tmp/ipykernel_171845/2343872101.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)


## Exploring the results

In [5]:
#adding IMACLIM direct emissions to the df
IMACLIM_emissions=extract_data(aggregation="IMACLIM_format")[6]

production_based_emissions["2050-IMACLIM"]=IMACLIM_emissions.drop("World",level=1)[2050].rename_axis(index=str.lower)*1e9
production_based_emissions["2015-IMACLIM"]=IMACLIM_emissions.drop("World",level=1)[2015].rename_axis(index=str.lower)*1e9

/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [6]:
diff_rapport =((production_based_emissions["2050-IMACLIM"]/production_based_emissions["2015-IMACLIM"])-(production_based_emissions["2050-Prod"]/production_based_emissions["2015"]))

for scenario in scenario_list:
    print(scenario,diff_rapport.loc[scenario].max())

INDC 8.268941087408166e-12
INDC2020_1000 8.035572207631958e-12
INDC2030_1000 7.099210108663101e-12
INDC2020_1600 8.144818153255073e-12
INDC2020_400 6.13709083552294e-12
INDC2030_1600 8.04800670550776e-12
baseline 9.425571434462654e-12


In [36]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region","sector"]).sum().loc["INDC"] 
Relative_error_2015=(Relative_error_2015["2015"]-Relative_error_2015["2015-IMACLIM"])/np.max([Relative_error_2015["2015"],Relative_error_2015["2015-IMACLIM"]],axis=0)
Relative_error_2015=Relative_error_2015*100
pd.concat([Relative_error_2015.loc[region] for region in Relative_error_2015.index.get_level_values("region").unique()],keys=Relative_error_2015.index.get_level_values("region").unique(),axis=1)

region,AFR,BRA,CAN,CHN,CIS,EUR,IND,JAN,MDE,RAL,RAS,USA
sector,,,,,,,,,,,,
Agriculture,-43.520474,36.645700,17.158575,-44.859290,-79.566081,-6.082898,46.911948,-59.574627,-55.962035,-17.970046,29.793671,-0.988413
Air,-45.172227,-23.270153,12.436130,71.661958,-1.527087,21.934999,45.944462,-43.838684,46.889270,-67.679364,39.297675,-5.631817
Coal,44.122147,27.782410,-88.077609,8.506734,-46.838304,-42.924087,96.470371,-22.246558,100.000000,100.000000,99.626550,-51.110549
Construction,84.668945,99.708036,76.663408,96.624980,70.970976,94.460861,95.967109,72.836376,83.099541,98.908128,96.687807,89.505142
Elec,-39.367526,72.760880,-56.968570,38.523421,-96.873208,-44.842949,25.343792,-13.061264,15.577006,-20.202585,7.368399,-19.935929
Gas,49.769182,81.839608,7.038038,97.653085,-7.778965,87.224283,68.865246,100.000000,-30.457425,-2.817599,31.503872,71.159567
Industry,-43.893401,4.640709,-50.285890,22.150305,-73.041125,-28.195911,-13.054238,-59.278069,37.025988,9.404065,12.147867,-52.534332
Liquid fuels,-91.631254,-76.275770,-37.110566,-43.896218,-55.277679,-77.231510,-4.899449,-81.704037,100.000000,100.000000,-60.917096,99.872532
Maritime,-89.689210,-70.072280,46.427379,70.225032,100.000000,41.189947,93.873600,40.498453,-25.642740,-70.193055,-23.795701,-42.158055


In [29]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region"]).sum().loc["INDC"]
Relative_error_2015["2015"]=Relative_error_2015["2015"]+model.iot.stressor_extension.F_Y.sum(axis=1,level=0).loc["CO2"]
Relative_error_2015_ratio=(Relative_error_2015["2015"]-Relative_error_2015["2015-IMACLIM"])/np.max([Relative_error_2015["2015"],Relative_error_2015["2015-IMACLIM"]],axis=0)
Relative_error_2015_ratio=Relative_error_2015_ratio*100
pd.DataFrame(Relative_error_2015_ratio)

,0
region,
AFR,-18.462489
BRA,26.513468
CAN,6.382565
CHN,45.768560
CIS,-51.092658
EUR,15.111379
IND,32.134522
JAN,-19.621153
MDE,30.733045


In [24]:
Relative_error_2015["2015"]+model.iot.stressor_extension.F_Y.sum(axis=1,level=0).loc["CO2"]

region
AFR    1.075643e+12
BRA    4.973495e+11
CAN    5.678739e+11
CHN    9.929628e+12
CIS    1.475975e+12
EUR    4.548643e+12
IND    2.118380e+12
JAN    1.628660e+12
MDE    2.189486e+12
RAL    1.286805e+12
RAS    3.212389e+12
USA    5.145479e+12
dtype: float64

In [3]:
import src.advance
raw_energy_tech=src.advance.extract_data("IMACLIM_format",return_tech_coef=True)

In [4]:
raw_energy_tech.loc[]

2015     2016     2017     2018  \
Scenario Sector                Region                                       
INDC     Elec                  AFR       3.445    3.676    3.914    4.155   
                               AFR       2.746    2.938    3.134    3.334   
                               BRA       1.521    1.544    1.568    1.596   
                               BRA       1.500    1.522    1.546    1.574   
                               CAN       2.806    2.873    2.936    2.995   
...                                        ...      ...      ...      ...   
baseline Terrestrial transport MDE       5.287    5.413    5.548    5.701   
                               RAL       8.090    8.388    8.673    8.968   
                               RAS       8.866    9.173    9.486    9.822   
                               USA      30.180   30.520   30.870   31.200   
                               World   109.200  111.700  114.100  116.500   

                                          2019     2020     2021     2022  \
Scenario Sector                Region                                       
INDC     Elec                  AFR       4.405    4.670    4.948    5.234   
                               AFR       3.543    3.764    3.996    4.236   
                               BRA       1.628    1.664    1.697    1.730   
                               BRA       1.606    1.642    1.674    1.707   
                               CAN       3.052    3.095    3.140    3.184   
...                                        ...      ...      ...      ...   
baseline Terrestrial transport MDE       5.878    6.065    6.268    6.479   
                               RAL       9.272    9.584    9.899   10.210   
                               RAS      10.180   10.550   10.920   11.300   
                               USA      31.550   31.910   32.280   32.660   
                               World   119.100  121.700  124.500  127.200   

                                          2023     2024  ...     2041  \
Scenario Sector                Region                    ...            
INDC     Elec                  AFR       5.540    5.864  ...   13.240   
                               AFR       4.492    4.766  ...   11.110   
                               BRA       1.768    1.812  ...    2.547   
                               BRA       1.746    1.789  ...    2.533   
                               CAN       3.228    3.272  ...    4.070   
...                                        ...      ...  ...      ...   
baseline Terrestrial transport MDE       6.714    6.964  ...   11.770   
                               RAL      10.530   10.870  ...   16.360   
                               RAS      11.710   12.140  ...   21.010   
                               USA      33.050   33.450  ...   41.010   
                               World   130.100  133.200  ...  189.400   

                                          2042     2043     2044     2045  \
Scenario Sector                Region                                       
INDC     Elec                  AFR      13.710   14.180   14.680   15.170   
                               AFR      11.520   11.940   12.370   12.810   
                               BRA       2.575    2.603    2.636    2.668   
                               BRA       2.562    2.591    2.625    2.658   
                               CAN       4.098    4.125    4.149    4.172   
...                                        ...      ...      ...      ...   
baseline Terrestrial transport MDE      12.070   12.370   12.670   12.970   
                               RAL      16.680   16.980   17.290   17.590   
                               RAS      21.660   22.320   23.010   23.690   
                               USA      41.490   41.950   42.410   42.860   
                               World   193.000  196.700  200.500  204.300   

                                          2046     2047     2048     2049  \
Scenario 

In [33]:
IMACLIM_emissions.drop("World",level=1)[2050].rename_axis(index=str.lower)

scenario  region  sector               
INDC      USA     Coal                      71.03000
                  Oil                      188.00000
                  Gas                       28.79000
                  Liquid fuels               0.06345
                  Elec                     301.90000
                                             ...    
baseline  RAL     Air                      280.50000
                  Maritime                  48.61000
                  Terrestrial transport    266.80000
                  Agriculture               48.42000
                  Industry                 439.80000
Name: 2050, Length: 1008, dtype: float64

In [28]:
import numpy as np
for scenario in scenario_list:
    dist=0
    dist+=np.sqrt(((production_based_emissions.loc[scenario,"2015"]-production_based_emissions.loc[scenario,"2015-IMACLIM"])**2).sum())
    print(scenario,dist)

INDC 3179847450552.0312
INDC2020_1000 3227317612158.8804
INDC2030_1000 3179847450552.0312
INDC2020_1600 3227317612158.8804
INDC2020_400 3227317612158.8804
INDC2030_1600 3179847450552.0312
baseline 3451761445905.605


region  sector               
AFR     Agriculture              2.280078e+10
        Air                      1.836116e+10
        Coal                     1.290314e+10
        Construction             8.225135e+10
        Elec                     3.669166e+11
                                     ...     
USA     Liquid fuels             1.535406e+11
        Maritime                 6.998342e+10
        Oil                      5.275347e+10
        Services                 5.364953e+11
        Terrestrial transport    6.018212e+10
Name: CO2, Length: 144, dtype: float64

In [23]:
IMACLIM_emissions[2015]

Scenario  Region  Sector               
INDC      USA     Coal                       33.87000
                  Oil                       199.90000
                  Gas                        81.93000
                  Liquid fuels                0.07918
                  Elec                     2147.00000
                                              ...    
baseline  World   Air                       900.10000
                  Maritime                  543.90000
                  Terrestrial transport    2494.00000
                  Agriculture               918.90000
                  Industry                 7298.00000
Name: 2015, Length: 1092, dtype: float64

AttributeError: 'IOSystem' object has no attribute 'stressor_extension'

In [51]:
scenario="INDC"
model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)

Problems on sums of columns  Series([], dtype: float64)


In [58]:
model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)
    

In [59]:
from pymrio.tools.iomath import calc_F_Y
F_Y=calc_F_Y(model.counterfactuals["IMACLIM_all_cons_change_2050_INDC"].iot.stressor_extension.S_Y,model.counterfactuals["IMACLIM_all_cons_change_2050_INDC"].iot.Y.sum())

In [60]:
F_Y

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     1.369166e+12   
CH4                                     4.337297e+11   
N2O                                     6.377476e+10   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            1.063049e+08                                     
CH4                                            1.259803e+06                                     
N2O                                            9.781622e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     2.834543e+09   
CH4                                     1.658924e+09   
N2O                                     2.236637e+08   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                                0.0                    0.0   
CH4                                0.0                    0.0   
N2O                                0.0                    0.0   
SF6                                0.0                    0.0   
HFC                                0.0                    0.0   
PFC                                0.0                    0.0   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2                       0.0                  0.0   
CH4                       0.0                  0.0   
N2O                       0.0                  0.0   
SF6                       0.0                  0.0   
HFC                       0.0                  0.0   
PFC                       0.0                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     2.441102e+11   
CH4                                     1.124609e+10   
N2O                                     4.881682e+09   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            1.104983e+08                                     
CH4                                            1.362729e+07                                     
N2O                                            1.502983e+05                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    

In [57]:
model.counterfactuals["IMACLIM_all_2050_INDC"].iot.stressor_extension.F_Y

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     3.116139e+11   
CH4                                     9.871424e+10   
N2O                                     1.451475e+10   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            2.419435e+07                                     
CH4                                            2.867235e+05                                     
N2O                                            2.226237e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     6.451246e+08   
CH4                                     3.775609e+08   
N2O                                     5.090450e+07   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                                0.0                    0.0   
CH4                                0.0                    0.0   
N2O                                0.0                    0.0   
SF6                                0.0                    0.0   
HFC                                0.0                    0.0   
PFC                                0.0                    0.0   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2                       0.0                  0.0   
CH4                       0.0                  0.0   
N2O                       0.0                  0.0   
SF6                       0.0                  0.0   
HFC                       0.0                  0.0   
PFC                       0.0                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     8.477625e+10   
CH4                                     3.905620e+09   
N2O                                     1.695344e+09   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            3.837461e+07                                     
CH4                                            4.732580e+06                                     
N2O                                            5.219661e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    

In [8]:
model.iot

In [1]:
final_technical_coef

NameError: name 'final_technical_coef' is not defined

In [7]:
production_based_emissions.loc["INDC","USA","Liquid fuels"]

2015            6.211744e+10
2050            2.969946e+10
2050-Prod       4.977711e+10
2050-Cons       7.718590e+10
2050-IMACLIM    6.345000e+07
2015-IMACLIM    7.918000e+07
Name: (INDC, USA, Liquid fuels), dtype: float64

In [24]:
model.iot.stressor_extension.F_Y

region                                           AFR  \
category Final consumption expenditure by households   
CO2                                     3.116139e+11   
CH4                                     9.871424e+10   
N2O                                     1.451475e+10   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            2.419435e+07                                     
CH4                                            2.867235e+05                                     
N2O                                            2.226237e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                \
category Final consumption expenditure by government   
CO2                                     6.451246e+08   
CH4                                     3.775609e+08   
N2O                                     5.090450e+07   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                         \
category Gross fixed capital formation Changes in inventories   
CO2                                0.0                    0.0   
CH4                                0.0                    0.0   
N2O                                0.0                    0.0   
SF6                                0.0                    0.0   
HFC                                0.0                    0.0   
PFC                                0.0                    0.0   

region                                              \
category Changes in valuables Exports: Total (fob)   
CO2                       0.0                  0.0   
CH4                       0.0                  0.0   
N2O                       0.0                  0.0   
SF6                       0.0                  0.0   
HFC                       0.0                  0.0   
PFC                       0.0                  0.0   

region                                           BRA  \
category Final consumption expenditure by households   
CO2                                     8.477625e+10   
CH4                                     3.905620e+09   
N2O                                     1.695344e+09   
SF6                                     0.000000e+00   
HFC                                     0.000000e+00   
PFC                                     0.000000e+00   

region                                                                                         \
category Final consumption expenditure by non-profit organisations serving households (NPISH)   
CO2                                            3.837461e+07                                     
CH4                                            4.732580e+06                                     
N2O                                            5.219661e+04                                     
SF6                                            0.000000e+00                                     
HFC                                            0.000000e+00                                     
PFC                                            0.000000e+00                                     

region                                                ...  \
category Final consumption expenditure by government  ...   
CO2                    